<a href="https://colab.research.google.com/github/Msurya-05/surya/blob/main/SURYA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install required libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread
!pip install requests

In [2]:
# Upload Gmail credentials.json file securely via Colab uploader
from google.colab import files
uploaded = files.upload()

Saving client_secret_638872432364-cb65r7uabc2tt96mprv86insuho20hc0.apps.googleusercontent.com.json to client_secret_638872432364-cb65r7uabc2tt96mprv86insuho20hc0.apps.googleusercontent.com.json


In [10]:
import os
import pickle
import base64
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

from google_auth_oauthlib.flow import Flow

flow = Flow.from_client_secrets_file(
    'client_secret_638872432364-cb65r7uabc2tt96mprv86insuho20hc0.apps.googleusercontent.com.json',
    scopes=SCOPES,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'
)
auth_url, _ = flow.authorization_url(prompt='consent')

print("Please visit this URL to authorize:")
print(auth_url)

# Paste authorization code
code = input("Enter authorization code: ")
flow.fetch_token(code=code)
creds = flow.credentials

# Finally build Gmail service object
service = build('gmail', 'v1', credentials=creds)


Please visit this URL to authorize:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=638872432364-cb65r7uabc2tt96mprv86insuho20hc0.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=FBZK0pZ2UmhYZi0THD7Hf8J9wH1PuP&prompt=consent&access_type=offline
Enter authorization code: 4/1AVMBsJhrAk5yIwueL2O-7wgZZsfnVQDd_VYSwix3668vLyzaj3DqBkjQWMc


In [13]:
# Fetch recent 5 emails for processing
results = service.users().messages().list(userId='me', maxResults=5).execute()
messages = results.get('messages', [])
email_bodies = []

for msg in messages:
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    payload = txt['payload']
    parts = payload.get("parts")

    email_text = None
    if parts:
        for part in parts:
            if part.get("mimeType") == "text/plain" or part.get("mimeType") == "text/html":
                if part.get("body") and part['body'].get('data'):
                    data = part['body'].get('data')
                    byte_code = base64.urlsafe_b64decode(data)
                    email_text = byte_code.decode("utf-8")
                    break # Found a text part, no need to check further
    elif payload.get("body") and payload['body'].get('data'):
        # Handle cases where the body is directly in the payload (less common for complex emails)
        data = payload['body'].get('data')
        byte_code = base64.urlsafe_b64decode(data)
        email_text = byte_code.decode("utf-8")


    if email_text:
        email_bodies.append(email_text)


print("✅ Successfully fetched", len(email_bodies), "emails for AI analysis.")

✅ Successfully fetched 5 emails for AI analysis.


In [22]:
import google.generativeai as genai
from google.colab import userdata
import os

# Load Gemini API Key securely from Colab Secrets
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

# Configure Gemini API
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Define the system prompt
system_prompt = """
You are an Enterprise Smart Notification Agent for campus automation.

Your task for each email:
1. Generate a 5-line summary.
2. Extract action deadlines and tasks.
3. Classify category: Academic, HR, Finance, IT, General.
4. Assign urgency: Critical, High, Normal.
5. Extract: Action required? Yes/No.

Output strictly in format:
Summary: <summary>
Category: <category>
Urgency: <urgency>
Action: <yes/no>
"""

# Load the Gemini model
model = genai.GenerativeModel(model_name="gemini-2.5-flash")

# Smart Notification Agent function
def smart_notification_agent(email_text):
    prompt = f"{system_prompt}\n\nEmail:\n{email_text}"
    response = model.generate_content(prompt)
    return response.text.strip()

# Loop over fetched emails (assume email_bodies is a list of email texts)
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Email {idx+1} =====")
    result = smart_notification_agent(email_text)
    print(result)


===== Email 1 =====
Summary: The password for your Google Account suryamukkala08@gmail.com was changed. If you did not initiate this change, you should recover your account immediately. A direct link is provided for account recovery. You can also review your security activity for more details. This is an important security notification about your Google Account.
Category: IT
Urgency: Critical
Action: Yes

===== Email 2 =====
Summary: Google detected a new sign-in on a Windows device.
This notification is for your suryamukkala08@gmail.com account.
If this sign-in was authorized by you, no action is required.
If not, you should check your account activity immediately.
Google will help secure your account if the activity is unauthorized.
Category: IT
Urgency: High
Action: Yes

===== Email 3 =====
Summary: Your Google account, suryamukkala08@gmail.com, has been successfully recovered.
This notification confirms the account's restored access.
If you suspect the lockout was due to unauthori

In [34]:
import gspread
from google.oauth2.service_account import Credentials

# Your service account file (already uploaded to Colab)
SERVICE_ACCOUNT_FILE = '/lucky-curve-446015-g0-105802ac77d2.json'  # Replace if you used a different filename

# Define Sheets API scope
SCOPES_SHEETS = ['https://www.googleapis.com/auth/spreadsheets']

# Authenticate using Service Account
sheet_creds = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES_SHEETS)

client = gspread.authorize(sheet_creds)

# Connect to your Google Sheet
spreadsheet_id = '12--ciLq5HmhbfpOLolMImnDMif_ppw0FBC2Dcmekb7M'  # Replace with your actual spreadsheet ID
sheet = client.open_by_key(spreadsheet_id).sheet1

# (Optional) Write header row if not already written
sheet.append_row(["Date", "Sender", "Subject", "Summary", "Category", "Urgency", "ActionRequired"])

{'spreadsheetId': '12--ciLq5HmhbfpOLolMImnDMif_ppw0FBC2Dcmekb7M',
 'updates': {'spreadsheetId': '12--ciLq5HmhbfpOLolMImnDMif_ppw0FBC2Dcmekb7M',
  'updatedRange': 'Sheet1!A1:G1',
  'updatedRows': 1,
  'updatedColumns': 7,
  'updatedCells': 7}}

In [54]:
import datetime

# Example values for testing (replace with real email content later)
# Note: In a real scenario, you would ideally get sender and subject from the fetched email metadata.
# date_today = datetime.datetime.now().strftime("%d %b") # Use full date for better tracking
# sender = "HR Dept"
# subject = "Salary Update"

# Loop through all fetched emails
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Processing Email {idx+1} =====")

    # Assuming you have fetched email metadata (like sender, subject, date) in the email fetching step
    # Replace with actual metadata from your fetched emails
    # For now, using placeholder values
    date_today = datetime.datetime.now().strftime("%Y-%m-%d") # Using YYYY-MM-DD format
    sender = f"Sender {idx+1}" # Placeholder
    subject = f"Subject {idx+1}" # Placeholder


    result = smart_notification_agent(email_text)
    print(result)

    # Parse the LLM output more robustly
    lines = result.strip().split('\n')
    summary, category, urgency, action = "", "", "", ""

    for line in lines:
        if line.startswith("Summary:"):
            summary = line.split(":", 1)[1].strip()
        elif line.startswith("Category:"):
            category = line.split(":", 1)[1].strip()
        elif line.startswith("Urgency:"):
            urgency = line.split(":", 1)[1].strip()
        elif line.startswith("Action:"):
            action = line.split(":", 1)[1].strip()

    # Insert into Google Sheet
    try:
        sheet.append_row([date_today, sender, subject, summary, category, urgency, action])
        print("✅ Successfully inserted data into Google Sheet.")
    except Exception as e:
        print(f"❌ Error inserting data into Google Sheet: {e}")


===== Processing Email 1 =====
Summary: Your Google Account password for suryamukkala08@gmail.com has been changed. This is a security alert regarding this account activity. If you did not make this change, you should immediately recover your account via the provided link. You can also review recent security activity on your Google Account. This email is to inform you of important account changes.
Category: IT
Urgency: Critical
Action: Yes
✅ Successfully inserted data into Google Sheet.

===== Processing Email 2 =====
Summary: Google detected a new sign-in on your account.
The new sign-in occurred on a Windows device.
If you recognize this activity, no action is needed.
If this sign-in was not authorized by you, secure your account.
Click the "Check activity" link to verify or take action.
Category: IT
Urgency: High
Action: Yes
✅ Successfully inserted data into Google Sheet.

===== Processing Email 3 =====
Summary:
Your Google account, suryamukkala08@gmail.com, has been successfully r